In [ ]:
!pip install transformers
!pip install torchmetrics
!pip install pytorch_lightning

# Section 1

In [ ]:
from google.colab import drive
from torch import nn
from transformers import BertTokenizer, BertModel
import torch
import csv
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW
from torchmetrics import Accuracy
import os
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import time
import math
import numpy as np
from transformers import DataCollatorForLanguageModeling
import pytorch_lightning as pl

In [ ]:
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/data'

train_data_path = f'{data_path}/train'
true_train_path = f'{train_data_path}/true.csv'
false_train_path = f'{train_data_path}/false.csv'

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

true_label = 1
false_label = 0

true_saved_model_path = '/content/drive/MyDrive/models/true_bert_lm.pt'
false_saved_model_path = '/content/drive/MyDrive/models/false_bert_lm.pt'

In [ ]:
class LMSpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet)
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return torch.tensor(input_ids)

    def __len__(self):
      return len(self.dataset)

class BertLM(pl.LightningModule):

    def __init__(self, class_name):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')
        self.epoch_number = 0
        self.class_name = class_name

    def forward(self, input_ids, labels):
        return self.bert(input_ids=input_ids,labels=labels)

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        labels = batch["labels"]
        outputs = self(input_ids=input_ids, labels=labels)
        loss = outputs[0]
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        super().training_epoch_end(outputs)
        mean_loss = 0
        n_batch  = len(outputs)
        for i in range(n_batch):
            mean_loss += outputs[i]['loss'].cpu().numpy() / n_batch
        print(f"End of epoch {self.epoch_number} with mean loss '{mean_loss}' on label {self.class_name}.", "fine_tuning")
        self.epoch_number += 1
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5)

class BertLMPred(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')

    def forward(self, input_ids, labels=None):
        return self.bert(input_ids=input_ids,labels=labels)

In [ ]:
MASK = '[MASK]'
SEP = '[SEP]'
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_batch(batch, tokenizer):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def get_init_text(seed_sentence, max_len, tokenizer, batch_size=1):
    batch = [seed_sentence + [MASK] * max_len + [SEP] for _ in range(batch_size)]
    return tokenize_batch(batch, tokenizer)

def untokenize_batch(batch, tokenizer):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def generate_step(out, gen_idx, temperature=None, top_k=0, 
                  sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    logits = out.logits[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx


def parallel_sequential_generation(seed_text, tokenizer, model, batch_size=1, 
                                   max_len=15, top_k=0, temperature=None, 
                                   max_iter=300, burnin=200, cuda=False, 
                                   print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, tokenizer, batch_size=batch_size)
    mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
    for ii in range(max_iter):
        kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        #if idxs is a single number
        if isinstance(idxs, int):
            idxs = [idxs]
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]
            
    return untokenize_batch(batch, tokenizer)


def generate(tokenizer, n_samples, class_name, model, seed_text="[CLS]", 
             batch_size=1, max_len=25, sample=True, top_k=100, temperature=1.0, 
             burnin=200, max_iter=500, cuda=False, print_every=1):
    # main generation function to call
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    start_time = time.time()
    for batch_n in range(n_batches):
        batch = parallel_sequential_generation(seed_text, tokenizer, model, 
                                               max_len=max_len, top_k=top_k,
                                               temperature=temperature, 
                                               burnin=burnin, max_iter=max_iter, 
                                               cuda=cuda, verbose=False)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time), "fine_tuning")
            start_time = time.time()
        
        sentences += batch
    return sentences

def standardize_sentence(sent):
    sentence = []
    current_word = sent[0]
    for i in range(1, len(sent)):
        token = sent[i]
        if(token[0:2] == '##'):
            current_word += token[2:]
        else:
            sentence.append(current_word)
            current_word = token
    sentence.append(current_word)
    return sentence

def fine_tune_LM(class_name, tokenizer, epochs, batch_size, 
                 train_paths, labels, save_url=None, mlm_prob=0.25, 
                 use_gpu=True):
  
    dataset = LMSpellCheckingDataset(tokenizer, train_paths, labels)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                    mlm=True, 
                                                    mlm_probability=mlm_prob)
    train_loader = DataLoader(dataset, 
                              batch_size=batch_size, 
                              collate_fn=data_collator)
    model = BertLM(class_name)
    #using CPU
    if use_gpu:
        trainer = pl.Trainer(max_epochs=epochs, 
                             checkpoint_callback=False, 
                             logger=False, 
                             gpus=1)
    else:
        trainer = pl.Trainer(max_epochs=epochs, 
                             checkpoint_callback=False, 
                             logger=False)
        
    print(f"Start fine tuning BERT masked LM on class {class_name}", "fine_tuning")
    trainer.fit(model, train_loader)
    if save_url is not None:
        print(f"Finished training. Saving model in {save_url}", "fine_tuning")
        torch.save(model.state_dict(), save_url)

def fine_tune(tokenizer):
    labels = ['false', 'true']
    bert_model_url = "/content/drive/MyDrive/models/"
    reports_dir = "/content/drive/MyDrive/reports/"
    if not os.path.exists(reports_dir):
        os.makedirs(reports_dir, exist_ok=True)
    labels = [item for item in labels]
    
    for label in labels:
        print('----------\n')
        print(f"Generating sentences for {label} label ...", "fine_tuning")
        model_url = os.path.join(bert_model_url, f"{label}_bert_lm.pt")
        model = BertLMPred()
        model.load_state_dict(torch.load(model_url))
        model = model.cuda()
        sentences = generate(tokenizer, 10, label, model, max_len=10, seed_text='[CLS]'.split(), cuda=True)
        out_url = reports_dir + f"{label}.txt"
        with open(out_url, 'w') as outf:
            for sent in sentences:
                sent = standardize_sentence(sent)
                outf.write(' '.join(sent).replace('[', '<').replace(']', '>'))
                outf.write("\\\\")
                outf.write("\n")
        print(f"Sentences for {label} saved to {out_url}.", "fine_tuning")
        print(sentences)
        print('----------\n')
        
fine_tune_LM('true', tokenizer, 20, 32, [true_train_path], [true_label], 
             save_url=true_saved_model_path)

fine_tune_LM('false', tokenizer, 20, 32, [false_train_path], [false_label], 
             save_url=false_saved_model_path)

fine_tune(tokenizer)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated

Start fine tuning BERT masked LM on class true fine_tuning


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 109 M 
-----------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.057   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Finished training. Saving model in /content/drive/MyDrive/models/true_bert_lm.pt fine_tuning


# Section 2

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/University/NLP/Final-Project/data'
# data_path = '/content/drive/MyDrive/data'

train_data_path = f'{data_path}/train'
true_train_path = f'{train_data_path}/true.csv'
false_train_path = f'{train_data_path}/false.csv'

test_data_path = f'{data_path}/test'
true_test_path = f'{test_data_path}/true.csv'
false_test_path = f'{test_data_path}/false.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Pytorch's nn module has lots of useful feature
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from torch.nn.utils.rnn import pad_sequence
import csv

def pad_batched_sequence(batch):
    tweets = []
    tweets_lengths = []
    labels = []
    for (tweet, label) in batch:
      tweets.append(torch.tensor(tweet).cuda())
      tweets_lengths.append(len(tweet))
      labels.append(label)

    tweets = pad_sequence(tweets, padding_value=0, batch_first=True).cuda()
    tweets_lengths = torch.tensor(tweets_lengths).cuda()
    labels = torch.tensor(labels).cuda()
    return tweets, tweets_lengths, labels

word_to_idx = {
  '<pad>': 0,
  '<start>': 1,
  '<stop>': 2
}
class SpellCheckingDataset(Dataset):

    def __init__(self, data_paths, labels, batch_size=32):
        self.dataset = []
        
        idx = 3
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                tokenized_tweet = ['<start>'] + word_tokenize(item[0]) + ['<stop>']
                for word in tokenized_tweet:
                  if (word not in word_to_idx):
                    word_to_idx[word] = idx
                    idx += 1
                self.dataset.append((tokenized_tweet, labels[i]))
        self.batch_size = batch_size
        self.vocab_size = len(word_to_idx)

    def __getitem__(self, idx):
        return [word_to_idx[w] for w in self.dataset[idx][0]], self.dataset[idx][1]

    def __len__(self):
      return len(self.dataset)

class LSTMNet(nn.Module):
    
    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout):
        
        super(LSTMNet,self).__init__()
        
        # Embedding layer converts integer sequences to vector sequences
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        
        # LSTM layer process the vector sequences 
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout,
                            batch_first = True
                           )
        
        # Dense layer to predict 
        self.fc = nn.Linear(hidden_dim * 2,output_dim)
        # Prediction activation function
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self,text,text_lengths):
        embedded = self.embedding(text)
        
        # Thanks to packing, LSTM don't see padding tokens 
        # and this makes our model better
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(),batch_first=True, enforce_sorted=False)
        
        packed_output,(hidden_state,cell_state) = self.lstm(packed_embedded)
        
        # Concatenating the final forward and backward hidden states
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)
        
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.sigmoid(dense_outputs)
        
        return outputs
    

In [ ]:

true_label = 1
false_label = 0

train_dataset = SpellCheckingDataset([true_train_path, false_train_path], 
                                          [true_label, false_label])

test_dataset = SpellCheckingDataset([true_test_path, false_test_path], 
                                          [true_label, false_label])

SIZE_OF_VOCAB = train_dataset.vocab_size
EMBEDDING_DIM = 100
NUM_HIDDEN_NODES = 64
NUM_OUTPUT_NODES = 1
NUM_LAYERS = 2
BIDIRECTION = True
DROPOUT = 0.2

In [ ]:
model = LSTMNet(SIZE_OF_VOCAB,
                EMBEDDING_DIM,
                NUM_HIDDEN_NODES,
                NUM_OUTPUT_NODES,
                NUM_LAYERS,
                BIDIRECTION,
                DROPOUT
               )



In [ ]:
import torch.optim as optim
model = model.cuda()
optimizer = optim.Adam(model.parameters(),lr=1e-4)
criterion = nn.BCELoss()
criterion = criterion.cuda()

In [ ]:
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model,iterator,optimizer,criterion):
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    model.train()
    
    for batch in iterator:
        
        # cleaning the cache of optimizer
        optimizer.zero_grad()
        
        text,text_lengths, labels = batch
        
        # forward propagation and squeezing
        predictions = model(text,text_lengths).squeeze()
        
        # computing loss / backward propagation
        loss = criterion(predictions.double(),labels.double())
        loss.backward()
        
        # accuracy
        acc = binary_accuracy(predictions,labels)
        
        # updating params
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    # It'll return the means of loss and accuracy
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
        

In [ ]:
def evaluate(model,iterator,criterion):
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    # deactivate the dropouts
    model.eval()
    
    # Sets require_grad flat False
    with torch.no_grad():
        for batch in iterator:
            text,text_lengths, labels = batch
            
            predictions = model(text,text_lengths).squeeze()
              
            #compute loss and accuracy
            loss = criterion(predictions.double(), labels.double())
            acc = binary_accuracy(predictions, labels)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
from torch.utils.data.dataloader import DataLoader

EPOCH_NUMBER = 15
train_loader = DataLoader(train_dataset, 
                          batch_size=32, 
                          shuffle=True,
                          drop_last=True, 
                          collate_fn=pad_batched_sequence)

test_loader = DataLoader(test_dataset, 
                          batch_size=32, 
                          shuffle=True,
                          drop_last=True, 
                          collate_fn=pad_batched_sequence)

for epoch in range(1,EPOCH_NUMBER+1):
    
    train_loss,train_acc = train(model,train_loader,optimizer,criterion)
    
    valid_loss,valid_acc = evaluate(model,test_loader,criterion)
    
    # Showing statistics
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()

	Train Loss: 0.692 | Train Acc: 52.11%
	 Val. Loss: 0.692 |  Val. Acc: 51.88%

	Train Loss: 0.669 | Train Acc: 58.39%
	 Val. Loss: 0.698 |  Val. Acc: 53.89%

	Train Loss: 0.632 | Train Acc: 63.48%
	 Val. Loss: 0.698 |  Val. Acc: 55.10%

	Train Loss: 0.589 | Train Acc: 68.44%
	 Val. Loss: 0.714 |  Val. Acc: 56.64%

	Train Loss: 0.542 | Train Acc: 72.23%
	 Val. Loss: 0.740 |  Val. Acc: 57.96%

	Train Loss: 0.497 | Train Acc: 75.29%
	 Val. Loss: 0.743 |  Val. Acc: 59.78%

	Train Loss: 0.457 | Train Acc: 78.12%
	 Val. Loss: 0.779 |  Val. Acc: 59.41%

	Train Loss: 0.414 | Train Acc: 80.82%
	 Val. Loss: 0.835 |  Val. Acc: 59.97%

	Train Loss: 0.380 | Train Acc: 82.61%
	 Val. Loss: 0.869 |  Val. Acc: 60.98%

	Train Loss: 0.344 | Train Acc: 84.58%
	 Val. Loss: 0.956 |  Val. Acc: 60.26%

	Train Loss: 0.310 | Train Acc: 86.54%
	 Val. Loss: 0.977 |  Val. Acc: 62.02%

	Train Loss: 0.278 | Train Acc: 88.12%
	 Val. Loss: 1.016 |  Val. Acc: 61.99%

	Train Loss: 0.249 | Train Acc: 89.44%
	 Val. Loss: 

# Section 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/data'

train_data_path = f'{data_path}/train'
true_train_path = f'{train_data_path}/true.csv'
false_train_path = f'{train_data_path}/false.csv'

test_data_path = f'{data_path}/test'
true_test_path = f'{test_data_path}/true.csv'
false_test_path = f'{test_data_path}/false.csv'

Mounted at /content/drive


In [ ]:


class TextClassificationModel(nn.Module):

    def __init__(self, embed_dim=768, num_class=1):
        super(TextClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased").cuda()
        self.fc = nn.Linear(embed_dim, num_class).cuda()
        self.sigmoid = nn.Sigmoid().cuda()

    def forward(self, input_ids, attention_masks):
        inputs = {
            "input_ids": input_ids.cuda(), 
            "attention_mask" : attention_masks.cuda()
        }
        outputs = self.bert(**inputs)
        output = outputs.last_hidden_state
        output = output[:,0,:]
        output = self.fc(output)
        return self.sigmoid(output)


In [ ]:
class SpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet)
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return input_ids, attention_mask, label

    def __len__(self):
      return len(self.dataset)

class LMSpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet)
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return torch.tensor(input_ids)

    def __len__(self):
      return len(self.dataset)

In [ ]:

def pad_batched_sequence(batch):
    
    input_ids = [torch.tensor(item[0]) for item in batch]

    attention_masks =  [torch.tensor(item[1]) for item in batch]

    input_ids = pad_sequence(input_ids, padding_value=0, batch_first=True)

    attention_masks = pad_sequence(attention_masks, 
                                   padding_value=0, 
                                   batch_first=True)

    labels = None
    
    if batch[0][2] is not None:
        labels = torch.tensor([[item[2]] for item in batch]).double().cuda()
    
    return input_ids.cuda(), attention_masks.cuda(), labels

class SpellCheckingTrainer():

  def __init__(self,
               model,
               train_dataset,
               save_data_path,
               batch_size=32,
               epochs=20,
               lr=0.001):
        
    self.model = model
    self.epochs = epochs
    self.batch_size = batch_size
    self.train_loader = DataLoader(train_dataset, 
                                   batch_size=batch_size, 
                                   shuffle=True, 
                                   drop_last=True, 
                                   collate_fn=pad_batched_sequence)
    self.save_path = save_data_path
    self.loss_function = nn.BCELoss()
    self.optimizer = torch.optim.Adam(list(self.model.parameters()), lr=lr)        
    self.accuracy = Accuracy(num_classes=1)

  def train_one_epoch(self, epoch_index):
    running_loss = 0.
    running_accuracy = 0.
    last_loss = 0.
    threshold = torch.tensor([0.5]).cuda()

    for i, data in tqdm.tqdm(enumerate(self.train_loader), 
                             total=len(self.train_loader)):
        # Every data instance is an input + label pair
        input_ids, attention_masks, labels = data

        # Zero your gradients for every batch!
        self.optimizer.zero_grad()

        # Make predictions for this batch
        outputs = self.model(input_ids, attention_masks)

        # Compute the loss and its gradients
        loss = self.loss_function(outputs.double(), labels.double())
        loss.backward()

        # Adjust learning weights
        self.optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        
        result = (outputs > threshold).float() * 1
        running_accuracy += torch.sum(result == labels) / self.batch_size
        
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            last_accuracy = running_accuracy / 10 # accuracy per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            print('  batch {} accuracy: {}'.format(i + 1, last_accuracy))
            running_loss = 0.
            running_accuracy = 0.

    return last_loss


  def train(self):
    epoch_number = 0
    for epoch in range(self.epochs):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        self.model.train(True)
        avg_loss = self.train_one_epoch(epoch_number)

        # We don't need gradients on to do reporting
        self.model.train(False)

        epoch_number += 1
    
    torch.save(self.model.state_dict(), self.save_path)
    
          
          

In [ ]:


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

true_label = 1
false_label = 0

true_train_dataset = SpellCheckingDataset(tokenizer, [true_train_path], [true_label])
false_train_dataset = SpellCheckingDataset(tokenizer, [false_train_path], [false_label])

test_dataset = SpellCheckingDataset(tokenizer, 
                                    [true_test_path, false_test_path],
                                    [true_label, false_label])

true_saved_model_path = '/content/drive/MyDrive/models/true_bert.berm_lm'
false_saved_model_path = '/content/drive/MyDrive/models/false_bert.berm_lm'

true_trainer = SpellCheckingTrainer(TextClassificationModel(), 
                                    true_train_dataset,
                                    true_saved_model_path)

false_trainer = SpellCheckingTrainer(TextClassificationModel(), 
                                     false_train_dataset,
                                     false_saved_model_path)
 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.trans

In [ ]:
true_trainer.train()
false_trainer.train()




EPOCH 1:


  2%|▏         | 10/403 [00:03<02:13,  2.95it/s]

  batch 10 loss: 0.0809702505817
  batch 10 accuracy: 0.90625


  5%|▍         | 20/403 [00:06<02:03,  3.09it/s]

  batch 20 loss: 9.74941587150786e-05
  batch 20 accuracy: 1.0


...

 74%|███████▍  | 300/403 [01:46<00:37,  2.76it/s]

  batch 300 loss: 1.4528633847100088e-07
  batch 300 accuracy: 1.0


 77%|███████▋  | 310/403 [01:50<00:33,  2.78it/s]

  batch 310 loss: 1.3597310888347206e-07
  batch 310 accuracy: 1.0


 79%|███████▊  | 317/403 [01:52<00:30,  2.85it/s]